In [10]:
import numpy as np

In [31]:
#   .txt
file_name = 'test.txt'

In [26]:
def deliverable_1(file_name, verbose=False):
    with open(file_name, 'r') as file:
        content = file.read()

        total_char = len(content)

        unique_char = set(content)
        num_unique_char = len(unique_char)

        average_bit = int(np.ceil(np.log(num_unique_char)))

        total_bit = average_bit * total_char

        if verbose:
            print(content, end='\n\n')
            print(unique_char, end='\n\n')

        print('Total number of characters: {} \nNumber of distinct characters: {} \nTotal number of bits needed to encode the text: {} \nAverage number of bits per character: {}'.format(
            total_char, num_unique_char, total_bit, average_bit))

In [27]:
deliverable_1(file_name)

I live in constant fear of the western descent towards fascism. But I digress.

The club is bumping. The ladies look good. The alcohol is flowing.

There is much pain in the word, but not in this room.

{'w', 'm', ' ', 'n', 'a', 'h', 'c', 'b', 'r', 'o', 'l', 'i', 'p', 't', 'B', 'd', 'u', 'f', 'v', '\n', 'e', 'T', 'g', 'I', 'k', '.', 's', ','}

Total number of characters: 201 
Number of distinct characters: 28 
Total number of bits needed to encode the text: 804 
Average number of bits per character: 4


In [39]:
def deliverable_2(file_name, verbose=False):
    with open(file_name, 'r') as file:
        content = file.read()

        total_char = len(content)

        num_unique_char = len(set(content))

        total_bit = int((1 + num_unique_char) * num_unique_char / 2)

        average_bit = round(total_bit / total_char, 2)

        print('Total numer of bits: {} \nAverage number of bits: {}'.format(
            total_bit, average_bit))

In [40]:
deliverable_2(file_name)

Total numer of bits: 406 
Average number of bits: 2.02
